## Permits for Arlington

In [ ]:
import  pandas     as   pd
import  geopandas  as  gpd
import  numpy      as   np


In [ ]:
def get_browser_page(url):

    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    import time
    
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized");

    browser = webdriver.Chrome("/usr/local/bin/chromedriver",options = chrome_options) 
    browser.get(url)

    browser.find_element_by_xpath('//input[@value = "Search"]').click()
    time.sleep(5)

    try:
        browser.find_element_by_partial_link_text('Show all').click()
        time.sleep(5)
    except:
        print("No show all button")

        
    page     =  browser . page_source
    permits  =  pd . read_html ( page )
    
    browser . close ( )
    
    return permits
        
    
def extract_ArlingtonMA_permits(year):
    import numpy as np
    #from datetime import datetime

    url = 'http://arlserver.town.arlington.ma.us/BuildingPermits/Select.pl?permittype=&issue=1/1/{year}&issue_thru=12/31/{year}'.format(year=year)
    
    permits = get_browser_page(url)

    permits = permits[1]   
    permits.columns = ['streetNum'] + list(permits.columns[1:])

    
    permits[['streetNum','Address']] = permits[['streetNum','Address']].ffill().replace(np.nan,' ', regex=True)
    #permits['asof']  =  datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    return permits


def permits_get_solar_norm(permits):

    mask = ~pd.isnull(permits.Description) & permits.Description.str.contains('Solar|solar')

    permits = permits[mask].copy()
    
    permits.Address=permits.Address.str.upper()
    
    for col in ['streetNum','Owner']:
        permits[col]=permits[col].astype(str)
    
    permits = permits[permits.Issued!='2001']  #85 	Oakland Ave 	2001 	B 	R 	23 	remodel kitchen 	Trinity Baptist Church 	David 	7000.0 	105.0 
    permits['Issued']=pd.to_datetime(permits['Issued'],format='%m-%d-%Y').dt.strftime('%Y-%m-%d')

    permits=permits.sort_values('Issued').reset_index(drop=True)

    if permits.Value.dtype==type(str):
        permits.loc[permits.Value.str.contains(' '),'Value']='0'
        permits.Value=permits.Value.astype(float)

    if permits.Fee.dtype==type(str):
        permits.loc[permits.Fee.str.contains(' '),'Fee']='0'
        permits.Fee=permits.Fee.astype(float)

    permits['installer'] = permits['Contractor'].str.lower().str.replace(' |;|,','', regex=True)
    permits['streetNum']  = permits['streetNum'].str.replace('\#','-', regex=True)
    
    permits = permits_norm(permits)

    summary = permits.groupby("installer").agg(
        {"installer":len,
         "Value":sum,
         "Fee":sum,
        }
    )
    summary.columns = ['installs','value','fee']
    
    aggregates = permits.groupby(['streetNum','Address']).agg(
    {
        "Permit":list,
        "Description":list,
        "Contractor":list,
        "Owner":max,
        "Value":sum,
        "Fee":sum,
        "Issued":[list,min]
    }).reset_index()

    aggregates.columns = ['streetNum','address','permits','descriptions','contractor','owner','permit_value','permit_fee','issued','effective_date']
    

    return aggregates, summary


def permits_norm(permits) :

    permits.Address=permits.Address . str . replace ( 'PARK$','PK', regex=True) \
                    . str . replace ( ' WAY$',' WY', regex=True) \
                    . str . replace ( '^BRATTLE TER$','BRATTLE TERR', regex=True) \
                    . str . replace ( '^APACHE TRL$','APACHE TR', regex=True) \
                    . str . replace ( '^FARMER CIR$|^FARMER\'S CIR$','FARMER`S CIR', regex=True) \
                    . str . replace ( '^GRANDVIEW RD$','GRAND VIEW RD', regex=True) \
                    . str . replace ( '^LOMBARD TER$','LOMBARD TERR', regex=True) \
                    . str . replace ( '^MOUNT VERNON ST$','MT. VERNON ST', regex=True) \
                    . str . replace ( '^N UNION ST$','NORTH UNION ST', regex=True) \
                    . str . replace ( '^PARK AVENUE EXT$','PARK AVE EXT', regex=True) \
                    . str . replace ( '^UPLAND RD$|^UPLAND RD W$','UPLAND RD WEST', regex=True) \
                    . str . replace ( '^WALNUT TER$','WALNUT TERR', regex=True) \
                    . str . replace ( '^WYMAN TER$','WYMAN TERR', regex=True) \
                    . str . replace ( '^SUMMER STREET PL$','SUMMER ST PL', regex=True) \
                    . str . replace ( '^RUSSELL TER$','RUSSELL TERR', regex=True) \
                    . str . replace ( '^LORRAINE TER$','LORRAINE TERR', regex=True) \
                    . str . replace ( '^LOWELL STREET PL$','LOWELL ST PL', regex=True) \
                    . str . replace ( '^PARK STREET PL$','PARK ST PL', regex=True) \
                    . str . replace ( '^WEST COURT TE$','WEST COURT TERR', regex=True) \
                    . str . replace ( '^LEE TER$','LEE TERR', regex=True) #\
#                     . str . replace ( '^$','', regex=True) \
#                     . str . replace ( '^$','', regex=True) \
    
    mask = (permits.Address == 'WASHINGTON ST') & (permits.streetNum=='73')
    permits.loc[mask,'streetNum'] = '71'
    mask = (permits.Address == 'CENTRAL ST') & (permits.streetNum=='10')
    permits.loc[mask,'streetNum'] = '8'
    mask = (permits.Address == 'ADAMS ST') & (permits.streetNum=='56-1')
    permits.loc[mask,'streetNum'] = '56'
    mask = (permits.Address == 'HERBERT RD') & (permits.streetNum=='84')
    permits.loc[mask,'streetNum'] = '82'
    mask = (permits.Address == 'BOWDOIN ST') & (permits.streetNum=='13')
    permits.loc[mask,'streetNum'] = '11'
    mask = (permits.Address == 'ALBERMARLE ST') & (permits.streetNum=='16')
    permits.loc[mask,'streetNum'] = '14'
    mask = (permits.Address == 'APPLETON PL') & (permits.streetNum=='377')
    permits.loc[mask,'Address'] = 'APPLETON ST'
    mask = (permits.Address == 'UPLAND RD WEST') & ((permits.streetNum=='11')|(permits.streetNum=='39'))
    permits.loc[mask,'Address'] = 'UPLAND RD'
    mask = ((permits.Address == 'UPLAND RD')|(permits.Address == 'UPLAND RD W')) & ((permits.streetNum=='45'))
    permits.loc[mask,'Address'] = 'UPLAND RD WEST'
    mask = (permits.Address == 'MASS AVE') & ((permits.streetNum=='626'))
    permits.loc[mask,'streetNum'] = '616'
    mask = (permits.Address == 'MASS AVE') & ((permits.streetNum=='869'))
    permits.loc[mask,'streetNum'] = '855'
    mask = (permits.Address == 'BROADWAY') & ((permits.streetNum=='37'))
    permits.loc[mask,'streetNum'] = '33'
    mask = (permits.Address == 'BROADWAY') & ((permits.streetNum=='297'))
    permits.loc[mask,'streetNum'] = '295'
    mask = (permits.Address == 'BROADWAY') & ((permits.streetNum=='144'))
    permits.loc[mask,'streetNum'] = '142'
    mask = (permits.Address == 'PARK AVE EXT') & ((permits.streetNum=='83')|(permits.streetNum=='82')|(permits.streetNum=='85'))
    permits.loc[mask,'streetNum'] = '99'
    mask = (permits.Address == 'NORTH UNION ST') & ((permits.streetNum=='60'))
    permits.loc[mask,'Address'] = 'EVERETT ST'
    permits.loc[mask,'streetNum'] = '187'
    
    return permits

## extract permits for multiple years
def extract_all_permits(data_dir , start_year = 2000 , end_year = 2022 ) :
    
    permits = pd . DataFrame ( )
    
    for year in range ( start_year , end_year ) :
        
        permits = permits . append ( extract_ArlingtonMA_permits ( year ) )

    solar_permits , contractor_leaders = permits_get_solar_norm ( permits )
    
    ##print(contractor_leaders.sort_values('installs',ascending=False)[0:30].to_markdown())
    
    solar_permits . to_csv ( data_dir + 'solar_permits.tsv' , sep = '\t' , index = False )
    permits . to_csv ( data_dir + 'permits.tsv' , sep = '\t' , index = False )

    return solar_permits, contractor_leaders

In [ ]:
data_dir = '/PATH/TO/YOUR/PERMITS/'
## Restore

solar_permits  =  pd . read_csv ( data_dir  +  'solar_permits.tsv' , sep = '\t' )
permits        =  pd . read_csv ( data_dir  +  'permits.tsv' , sep = '\t' )

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

summ = permits.groupby(permits.Issued.str[-4:]).agg({
    'Issued':len,
    'Value':sum,
    'Fee':sum
})
plt.figure(figsize=(6, 4), dpi=150)
plt.ylabel('Permitted Value')

#ax1 = summ["Issued"].plot(color='blue', grid=True, label='Issued')
ax2 = summ["Value"].plot(color='red', grid=True, label='Value')
ax3 = summ["Fee"].plot(color='green', grid=True, secondary_y=True, label='Fees')

#h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
h3, l3 = ax3.get_legend_handles_labels()
plt.xlabel('Fiscal Year')


plt.legend(h2+h3, l2+l3, loc=2)
plt.show()


In [ ]:
print('Permits,{installs},{first},{last}'.format(installs=len(solar_permits),first=solar_permits.effective_date.min(),last=solar_permits.effective_date.max()))

In [ ]:
def get_massgis_structures_assessor_shapefiles(data_dir = '/data/L3/L3_SHP_M010_Arlington/', town_code=10):

    url          =  'http://download.massgis.digital.mass.gov/shapefiles/structures/structures_poly_{town_code}.zip'
    structures   =  gpd . read_file ( url . format ( town_code = town_code ) ) . to_crs ( "EPSG:4326" )


    ##manual file download and merge shape and database file together
    url   =  data_dir  +  'M010TaxPar_CY21_FY21.shp'
    shp   =  gpd . read_file ( url ) . to_crs ( "EPSG:4326" )

    url   =  data_dir + 'M010Assess_CY21_FY21.dbf'
    dbf   =  gpd . read_file ( url ) . drop ( 'geometry' , axis = 1 ) . copy ( )  ## messed up geometry field all None

    combo  =  shp . merge ( dbf , on = ['TOWN_ID','LOC_ID'] , how = 'outer' , indicator = True )
    combo  =  combo [ combo . _merge == 'both' ] . reset_index ( drop = True )   ##63 no matches; water, roads, etc.
    combo  =  combo . drop ( '_merge' , axis = 1 )

    return combo, structures



In [ ]:
def merge_gis_permits ( solar_permits ) :
    
    assessor, structures = get_massgis_structures_assessor_shapefiles ( )
    
    solar_systems = solar_permits . merge ( 
        assessor , 
        right_on   =  [ 'ADDR_NUM' , 'FULL_STR' ] ,
        left_on    =  [ 'streetNum' , 'address' ] ,
        how        =    'outer' ,
        indicator  =  True
    )

    solar_systems_wo_geometry = solar_systems [ solar_systems . _merge == 'left_only' ]

    solar_systems = gpd . GeoDataFrame ( solar_systems [ solar_systems . _merge == 'both' ] ) . to_crs ( "EPSG:4326" ) . reset_index ( drop = True )

    solar_systems [ 'tokenId' ] = np . int64

    for idx in range ( len ( solar_systems ) ) :

        mask = structures . intersects ( solar_systems . loc [ idx , 'geometry' ] )   ##intersects Vs within finds all structures

        if len ( structures [ mask ] ) > 0 :
            
            foo = structures [ mask ] [ structures [ mask ] . AREA_SQ_FT == structures [ mask ] . AREA_SQ_FT . max ( ) ]

            center    =  ( foo . loc [ mask , 'geometry' ] ) .to_crs('+proj=cea') . centroid . to_crs ( foo . crs )
            solar_systems . loc [ idx , 'geometry' ] =  ( foo . geometry ) . values ## NB centroid returns lon / lat
            
        else :
            
            print ( idx , 'failed' )  ##using centroid of plot instead of structure
            
    return solar_systems , solar_systems_wo_geometry

In [ ]:
solar_systems , solar_systems_wo_geometry = merge_gis_permits ( solar_permits )